<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

### Introduction

In this exercise, I will import and use datasets that have the borough and neighborhood of Toronto. Also, I will use the Foursquare API to explore neighborhoods in Toronto. Use the explore function to group the neighborhoods into clusters, using the k-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.


Download the dependencies that we will need.

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2018.12=py37_0
  - defaults/win-64::blaze==0.11.3=py37_0
  - defaults/win-64::numba==0.41.0=py37hf9181ef_0
done

## Package Plan ##

  environment location: C:\Users\KIMTEOT\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           5 KB
    anaconda-custom            |           py37_1           2 KB
    ca-certificates-2019.11.28 |       hecc5488_0         182 KB  conda-forge
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    liblief-0.9.0              |       ha925a31_2         9.7 MB
    openssl-1.1.1d             |       hfa6e2cd_0         4.7 MB  conda-forge
    powe

DEBUG menuinst_win32:__init__(196): Menu: name: 'Anaconda${PY_VER} ${PLATFORM}', prefix: 'C:\Users\KIMTEOT\Anaconda3', env_name: 'None', mode: 'user', used_mode: 'user'
DEBUG menuinst_win32:create(320): Shortcut cmd is %windir%\System32\WindowsPowerShell\v1.0\powershell.exe, args are ['-ExecutionPolicy', 'ByPass', '-NoExit', '-Command', '"& \'C:\\Users\\KIMTEOT\\Anaconda3\\shell\\condabin\\conda-hook.ps1\' ; conda activate \'C:\\Users\\KIMTEOT\\Anaconda3\' "']


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\KIMTEOT\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.0.1-py_0
  branca             conda-forge/noarch::branca-0.4.0-py_0
  folium             conda-forge/noarch::folium-0.5.0-py_0
  vincent            conda-for

In [31]:
# Reading in data from local directory
df = pd.read_csv('C:/Users/KIMTEOT/Documents/Python Notebooks/Toronto.csv')
df.shape


(287, 3)

In [37]:
# droping rows with unassigned Boroughs
df_filtered = df[df[['Borough']].ne('Not assigned').all(axis=1)]
df_filtered.shape


(210, 3)

#### By dropping rows with unassigned Boroughs, lost 77 records

In [40]:
# Aggregating neighbourhoods by Bouroughs
nbh = df_filtered.groupby('Postcode').aggregate({'Borough': 'first', 'Neigbourhood': ','.join})
print(nbh.head())

              Borough                          Neigbourhood
Postcode                                                   
M1B       Scarborough                         Rouge,Malvern
M1C       Scarborough  Highland Creek,Rouge Hill,Port Union
M1E       Scarborough       Guildwood,Morningside,West Hill
M1G       Scarborough                                Woburn
M1H       Scarborough                             Cedarbrae


In [46]:
#Assigning the condition where the neighbourhood was unassigned to "True"
condition = (nbh["Borough"] != "Not assigned") & (nbh["Neigbourhood"] == "Not assigned")

# replace the value in the neighbourhood column with the value in the Borough when the condition is met.
nbh.loc[condition, "Neigbourhood"] = nbh.loc[condition, "Borough"]

nbh

,Borough,Neigbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"


In [47]:
# display number of rows and columns in dataframe
nbh.shape

(103, 2)

In [49]:
nbh.to_csv('C:/Users/KIMTEOT/Documents/Python Notebooks/nbh.csv', index=True)